In [11]:
import pandas as pd
import fool
import json

In [3]:
df_base = pd.read_csv("../data/baseinfo.csv",header=None)
df_attr = pd.read_csv("../data/atterinfo.csv",header=None)
df_qa = pd.read_csv("../data/qadata.csv",header=None)
df_base.columns, df_attr.columns, df_qa.columns = ["id","name","intro"],["id","name","key","values"],["id","name","question","answer"]

In [4]:
df_base.head(3)

,id,name,intro
0,00059a79db8982df0e75498e5b0b44bc,宁豇3号,宁豇3号是以之豇28-2和白豇2号的优良单株为亲本，经杂交系选育而成，是继宁豇1号、宁豇2号...
1,0006dc509671d4cf93940b5b1d0e67b3,优果网,优果网由上海汇果电子科技有限公司投资，于2007年正式上线运营，是国内最早从事水果直销的电子...
2,000bcacb826926c549e97d53ca7549d8,水毛花,水毛花，中药名。为莎草科植物水毛花Schoenoplectus mucronatus（L．）...


In [5]:
df_attr.head(3)

,id,name,key,values
0,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,中文学名,美丽异木棉
1,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,别名,美人树、美丽木棉、丝木棉
2,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,拉丁学名,Ceiba speciosa St.Hih.


In [6]:
df_qa.head(3)

,id,name,question,answer
0,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,形态特征,/美丽异木棉 编辑\n\n\n\t\t\t\n\t\t\n \n 美丽异木棉\n落叶乔木，高...
1,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,生态习性,/美丽异木棉 编辑\n\n\n\t\t\t\n\t\t美丽异木棉性喜光而稍耐阴，喜高温多湿气...
2,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,分布范围,/美丽异木棉 编辑\n\n\n\t\t\t\n\t\t美丽异木棉原产于南美洲，热带地区多有栽...


In [9]:
"问题个数：%s"%len(df_qa.question.drop_duplicates()), "属性个数：%s"%len(df_attr.key.drop_duplicates()),"所有农作物的个数：%s"%(df_base.shape[0])

('问题个数：4434', '属性个数：1230', '所有农作物的个数：5114')

In [8]:
df_attr[df_attr.key=="分布区域"]

,id,name,key,values
12,6cc9847077aae01d5c7904ea895c196e,美丽异木棉,分布区域,原产南美，中国多地城市广泛栽培
36,a7e71c32737c4e49616576844f60cd9b,茴香菖蒲,分布区域,四川省、湖南省、湖北省、贵州省、重庆、云南省、广西省、广东省
62,ea0c6559ba52aedef781bd18c176a1a8,腺绒杜鹃,分布区域,云南
80,7af61f2032c22d5c22011d40f6892ec9,楝叶吴萸,分布区域,"台湾,福建,广东,海南,广西,云南"
95,dab7070fb6b584c2f834a39add703744,水蜡,分布区域,原产于我国中南地区，现北方各地广泛栽培，日本也有分布。
...,...,...,...,...
42368,0b5182a12fead5cbd537f49396a46851,银耳,分布区域,中国
42386,42e5d5374140baac66f7448bf0d56b37,海鲜菇,分布区域,北温带
42401,b222d368b97fad51acbcf19693437ed5,油茶[山茶科植物],分布区域,中国从长江流域到华南各地广泛栽培
42419,59b6d901e6e04f8d222e26dea3706ec8,莆田龙眼,分布区域,莆田


In [62]:
df_attr[df_attr.key=="2"]

,id,name,key,values
28916,4850f81394483c656dfc1d99efbe3c52,咖喱蔬菜,2,炒咖喱卤或咖喱粉时一定要用小火


In [67]:
import ahocorasick

def make_AC(AC, word_set):
    for word in word_set:
        AC.add_word(word,word)
    return AC

def test_ahocorasick():
    '''
    ahocosick：自动机的意思
    可实现自动批量匹配字符串的作用，即可一次返回该条字符串中命中的所有关键词
    '''
    key_list = ["苹果", "香蕉", "梨", "橙子", "柚子", "火龙果", "柿子", "猕猴挑"]
    AC_KEY = ahocorasick.Automaton()
    AC_KEY = make_AC(AC_KEY, set(key_list))
    AC_KEY.make_automaton()
    test_str_list = ["我最喜欢吃的水果有：苹果、梨、香蕉、还有烤肉香蕉水", "我也喜欢吃香蕉，但是我不喜欢吃梨"]
    for content in test_str_list:
        name_list = set()
        for item in AC_KEY.iter(content):#将AC_KEY中的每一项与content内容作对比，若匹配则返回
            name_list.add(item[1])
        name_list = list(name_list)
        if len(name_list) > 0:
            print(content, "--->命中的关键词有：", "\t".join(name_list))
if __name__ == "__main__":
    test_ahocorasick()

我最喜欢吃的水果有：苹果、梨、香蕉、还有烤肉香蕉水 --->命中的关键词有： 苹果	梨	香蕉
我也喜欢吃香蕉，但是我不喜欢吃梨 --->命中的关键词有： 梨	香蕉


In [68]:

fool.analysis("我最喜欢吃的水果有：苹果、梨和香蕉还有芭乐")

([[('我', 'r'),
   ('最', 'd'),
   ('喜欢', 'vi'),
   ('吃', 'v'),
   ('的', 'ude'),
   ('水果', 'n'),
   ('有', 'vyou'),
   ('：', 'wm'),
   ('苹果', 'n'),
   ('、', 'wn'),
   ('梨', 'n'),
   ('和', 'c'),
   ('香蕉', 'n'),
   ('还有', 'v'),
   ('芭乐', 'n')]],
 [[(10, 13, 'company', '苹果'), (19, 22, 'company', '芭乐')]])

In [69]:
key_list = ["苹果", "香蕉", "梨", "橙子", "柚子", "火龙果", "柿子", "猕猴挑"]
test_str_list = ["我最喜欢吃的水果有：苹果、梨、香蕉、还有烤肉香蕉水", "我也喜欢吃香蕉，但是我不喜欢吃梨"]
for key in key_list:
    if key in test_str_list[0]:
        print(key)

苹果
香蕉
梨


In [21]:
words, ners = fool.analysis("原产于我国中南地区，现北方各地广泛栽培，日本也有分布。")
ners[0].sort(key=lambda x:x[0], reverse=True)
for start, end, ner_type, ner_name in ners[0]:
    print(start,'***********',end,'**********',ner_type,'*************',ner_name)

20 *********** 23 ********** location ************* 日本
5 *********** 10 ********** location ************* 中南地区
